In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14594803
paper_name = 'davis_kaplan_kaplan_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Copy of Kaplan Experimental diploid homozygous deletion srceen Release 1 and 2 sort sheet(3).xlsx', sheet_name='Exper. Results 1 & 2 sort sheet')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4793 x 92


In [7]:
original_data.head()

,Unnamed: 0,Unnamed: 1,ORF,Plate #,Unnamed: 4,Unnamed: 5,30° BPS 2.5 none,30° BPS 2.5 50µM Cu,30° BPS 2.5 100µM Fe,30° BPS 2.5 2mM Fe,...,37° YPD 100µM Fe.1,37° YPD 2mM Fe.1,37° YPgal none.1,37° YPgal 50µM Cu.1,37° YPgal 100µM Fe.1,37° YPgal 2mM Fe.1,37° YPGE none.1,37° YPGE 50µM Cu.1,37° YPGE 100µM Fe.1,37° YPGE 2mM Fe.1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,YAL068C,301.0,A,2.0,1,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,SEO1,YAL067C,301.0,A,3.0,1,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,YAL066W,301.0,A,4.0,1,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,YAL065C,301.0,A,5.0,1,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
original_data['orf'] = original_data[' ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
typo_fixes = pd.read_csv('extras/typo_fixes.txt', sep='\t', header=None)
typo_fixes_dict = {row[0]: row[1] for index, row in typo_fixes.iterrows() }
original_data['orf'] = original_data['orf'].apply(lambda x: typo_fixes_dict[x] if x in typo_fixes_dict.keys() else x)

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             Unnamed: 0 Unnamed: 1       ORF  Plate # Unnamed: 4  Unnamed: 5  \
index_input                                                                    
0                   NaN        NaN       NaN      NaN        NaN         NaN   
347               347.0        NaN    YMR41W    304.0          F         6.0   
4120             4120.0        NaN  YCL026C-    342.0          F        12.0   
4121             4121.0        NaN  YCR028C-    342.0          G         1.0   
4238             4238.0        NaN  YLR390W-    344.0          A         1.0   
4283             4283.0        NaN  YLR390W-    344.0          D        11.0   
4549             4549.0        NaN  YER119C-    347.0          C         6.0   

            30° BPS 2.5    none 30° BPS 2.5 50µM Cu 30° BPS 2.5 100µM Fe  \
index_input                                                                
0                           NaN                 NaN                  NaN   
347                           2                   3

In [13]:
original_data = original_data.loc[t,:]

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
data_cols = [c for c in original_data.columns.values if c.startswith('3')]

In [16]:
original_data = original_data[data_cols].copy()

In [17]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(4673, 80)

In [20]:
cols = [c.strip('.1').replace('°','C').replace('µ','u').replace(' ','') for c in original_data.columns.values]

# Load dataset ids

In [21]:
dt = pd.read_csv('extras/dataset_ids.txt', sep='\t', header=None)

In [22]:
dt.head()

,0,1
0,30° BPS 2.5 none,483
1,30° BPS 2.5 50µM Cu,5185
2,30° BPS 2.5 100µM Fe,5186
3,30° BPS 2.5 2mM Fe,5187
4,30° BPS 5 none,5188


In [23]:
dt.shape

(40, 2)

In [24]:
dt[0] = dt[0].apply(lambda x: x.replace('°','C').replace('µ','u').replace(' ',''))

In [25]:
dt.set_index(0, inplace=True)

In [26]:
dt.head()

,1
0,
30CBPS2.5none,483
30CBPS2.550uMCu,5185
30CBPS2.5100uMFe,5186
30CBPS2.52mMFe,5187
30CBPS5none,5188


In [27]:
dataset_ids = [dt.loc[x,1] for x in cols]

In [28]:
len(dataset_ids)

80

In [29]:
original_data.columns = dataset_ids

In [30]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

In [31]:
## Exclude the datasets with < 1000 tested genes
num_tested = np.sum(original_data.notnull(), axis=0)

In [32]:
original_data = original_data.loc[:, num_tested >= 1000]

In [33]:
original_data.shape

(4673, 30)

# Prepare the final dataset

In [34]:
data = original_data.copy()

In [35]:
dataset_ids = original_data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [36]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [37]:
data.head()

dataset_id,483,5185,5186,5188,5189,5190,5193,5194,5195,5197,...,5212,5213,5214,5215,5268,5269,5270,5271,5272,5274
data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,,,
YAL002W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,2.0,2.0,3.0,5.0,4.0,4.0,4.0,5.0,4.0
YAL004W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,3.0,3.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0
YAL005C,1.0,5.0,4.0,3.0,4.0,5.0,4.0,4.0,4.0,3.0,...,3.0,2.0,2.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0
YAL007C,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,3.0,3.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0
YAL008W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,2.0,3.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0


## Subset to the genes currently in SGD

In [38]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [39]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,483,5185,5186,5188,5189,5190,5193,5194,5195,5197,...,5212,5213,5214,5215,5268,5269,5270,5271,5272,5274
,data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,2.0,2.0,3.0,5.0,4.0,4.0,4.0,5.0,4.0
1863,YAL004W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,3.0,3.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0
4,YAL005C,1.0,5.0,4.0,3.0,4.0,5.0,4.0,4.0,4.0,3.0,...,3.0,2.0,2.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0
5,YAL007C,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,3.0,3.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0
6,YAL008W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,2.0,3.0,3.0,5.0,4.0,4.0,4.0,5.0,5.0


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,483,5185,5186,5188,5189,5190,5193,5194,5195,5197,...,5212,5213,5214,5215,5268,5269,5270,5271,5272,5274
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.016918,-0.018482,-0.017766,0.917252,0.90753,0.967422,0.01115,0.008345,1.137841,0.008822
1863,YAL004W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.854336,0.896355,0.861652,0.917252,0.90753,0.967422,0.01115,0.008345,1.137841,0.882246
4,YAL005C,1.0,5.0,4.0,3.0,4.0,5.0,4.0,4.0,4.0,3.0,...,0.016918,-0.018482,-0.017766,0.917252,0.90753,0.967422,0.01115,0.008345,1.137841,0.882246
5,YAL007C,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.854336,0.896355,0.861652,0.917252,0.90753,0.967422,0.01115,0.008345,1.137841,0.882246
6,YAL008W,1.0,5.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.854336,-0.018482,0.861652,0.917252,0.90753,0.967422,0.01115,0.008345,1.137841,0.882246


# Print out

In [43]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
from IO.save_data_to_db3 import *

In [45]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 14594803...


  0%|          | 0/30 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 30/30 [03:37<00:00,  7.25s/it]

Updating the data_modified_on field...
